In [165]:
import pandas as pd
df1 = pd.read_csv("vancouver_sample_data.xlsx - All listings.csv", usecols=[0,1,2,5])
df1.head(5)

,unique_host_id,internal_listing_id,external_property_id,neighborhood_name
0,air|3076262,air10001094,8IDQ97ApesVEG4mp,Fairview/Kitsilano/West Point Grey
1,air|6051853,air10003935,orq0PPposlfBQDTT,Riley Park/Renfrew-Collingwood/Kensington-Ceda...
2,air|517812,air10022535,wCeAQRws-bUzf3Xo,Hastings-Sunrise/Grandview-Woodland/Strathcona...
3,air|6051853,air10023231,y0bj34c7RwTxxhwI,Riley Park/Renfrew-Collingwood/Kensington-Ceda...
4,air|6051853,air10023601,++UeaQseWegvc0Wx,Riley Park/Renfrew-Collingwood/Kensington-Ceda...


In [15]:
df1.unique_host_id.count() # Total records

6376

In [19]:
un = df1.unique_host_id.value_counts()
# how many unique hosts are there in Vancouver?
un.count() # wast majority has only one record un.head(1000)

4948

In [42]:
df1.neighborhood_name.unique()

array(['Fairview/Kitsilano/West Point Grey',
       'Riley Park/Renfrew-Collingwood/Kensington-Cedar Co',
       'Hastings-Sunrise/Grandview-Woodland/Strathcona/Dow',
       'Shaughnessy/South Cambie/Fairview/Mount Pleasant',
       'West End/Downtown', 'Marpole/Kerrisdale',
       'Riley Park/Oakridge/Marpole/Sunset/Victoria-Fraser',
       'West End/English Bay',
       'Dunbar-Southlands/Arbutus-Ridge/Shaughnessy/South',
       'West End/Stanley Park'], dtype=object)

In [50]:
name_groups = df1.unique_host_id.groupby(df1['neighborhood_name'])
for name, group in name_groups: 
    # print the name of the regiment
    print(name)
    # print the data of that regiment
    print(group.value_counts().count())

Dunbar-Southlands/Arbutus-Ridge/Shaughnessy/South
256
Fairview/Kitsilano/West Point Grey
765
Hastings-Sunrise/Grandview-Woodland/Strathcona/Dow
767
Marpole/Kerrisdale
74
Riley Park/Oakridge/Marpole/Sunset/Victoria-Fraser
342
Riley Park/Renfrew-Collingwood/Kensington-Cedar Co
912
Shaughnessy/South Cambie/Fairview/Mount Pleasant
427
West End/Downtown
1366
West End/English Bay
165
West End/Stanley Park
55


In [96]:
# in case hosts and propertiese are not chained in clusters
# and don't have obscure values like no-host-ever
u_props = df1.groupby([ df1['external_property_id'], df1['unique_host_id']], as_index=False)
p_clusters = {}
h_clusters = {}
groups = []
for name, group in u_props:
    if(name[0] in p_clusters or name[1] in h_clusters):
        continue
    p_clusters[name[0]] = True
    h_clusters[name[1]] = True
    groups.append( name[0] + name[1] )
len(groups)

4789

In [166]:
# longer calculation in case there is chained clusters
# without use of the groupby (since groupby doesn't really help in our case)

#df = pd.DataFrame({'h':['a','a','b','c', 'a', 'a','c', 'f', 'b'], 'p':['x','w','y','v','v','w','r','r','a']}, columns = ['h','p'])

t = {}
clusters = {}
df1 = df1.fillna('missing')
for row in df1.itertuples():
    #print('-------')
    h0 = row.unique_host_id
    p = row.external_property_id
    if(h0=='missing' and p=='missing'):
        continue
    # for obscure values like properties without hosts and vice versa
    if(h0=='missing'):
        h0='missing'+p
    if(p=='missing'):
        p='missing'+h0
        
    if(h0 in t):
        h = t[h0]
    else:
        h=h0
    #print('(',h0,')',h,p)
    if(p not in t and h not in clusters):
        clusters[h] = True
        t[p] = h
        #print('new',h,p)
    elif(p not in t):
        t[p] = h
        #print('translation', p, h)
    elif(t[p] != h):
        print(h, t[p]) # print chained links
        #print("Add cluster:", h)
        clusters[h] = True
        dh = t[p]
        #print("Delete cluster:", dh)
        t[dh] = h
        #print('translation', dh, h)
        if(dh in clusters):
            del clusters[dh]
        for k in t:
            if ( t[k]==dh ):
                #print('translation ', k, '=', dh, ' -> ', h)
                t[k] = h
len(clusters)


air|21409576 air|20083932
air|52175189 air|30899
air|61551031 air|52175189
air|33243357 air|61551031
air|10981379 air|19139775
air|57171547 air|33243357
air|3987457 air|51998168
air|7336501 air|57171547
air|7232476 air|7336501
air|12405208 air|15143475
air|71002147 air|7232476
air|52151 air|10981379
air|7341876 air|21409576
air|71002147 air|68584515
air|61444830 air|71002147
air|6038958 air|61444830
air|90547736 air|7341876
air|16913110 air|52151
air|25567962 air|96071102
air|97168167 air|40603403
air|100693624 air|64108936
air|100700087 air|100693624
air|100803661 air|48371582
air|100702458 air|100803661
air|100287076 air|97168167
air|101110378 air|6038958
air|101122031 air|101110378
air|101126440 air|101122031
air|101276905 air|101126440
air|25647587 air|90547736
air|101998933 air|11472769
air|8442460 air|25647587
air|103009834 air|100702458
air|106428340 air|101276905
air|86003127 air|45929065
air|57805323 air|8442460
air|103889890 air|103009834
air|7796642 air|106428340
air|3082410

4813